In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

```markdown
# 🍜 AI Food Recommender: What to Eat & Where to Have It

## 🧠 Project Objective

The objective of this project is to develop an **AI agent** that can recommend **what to eat** and **where to have it** based on:

- User preferences
- Dietary restrictions
- Time of day (e.g., breakfast, lunch, dinner)
- Current weather condition
- User's current location and destination

This system will help users discover new **restaurants, hawkers, or food locations**, and make more personalized and informed decisions about their meals.

---

## 📌 Project Components

### 1. User Input Module
Collects the following:
- Preferred taste/flavor (e.g., spicy, savory, sweet)
- Dietary restrictions (e.g., vegetarian, halal, no peanuts)
- Time of day (breakfast, lunch, dinner)
- Current location (e.g., Bangsar)
- Destination (optional, e.g., heading to Sunway)
- Food type preference (optional: street food, café, healthy, quick bites)

---

### 2. Context-Aware Fetchers
- **Weather API** (stub or simulated): Fetch current weather to influence food types (e.g., soup on rainy days).
- **Time logic**: Suggest meals appropriate for breakfast, lunch, or dinner.
- **Place discovery module**:
  - Simulated or real API (e.g., Google Maps, Foursquare)
  - Suggest restaurants/hawkers near the current location or destination

---

### 3. LLM-Based Recommendation Engine
- Uses prompt-based logic to generate recommendations from:
  - User inputs
  - Current weather
  - Time of day
  - Nearby food options (filtered list)
- Outputs:
  - **What to eat** (dish suggestion)
  - **Where to eat** (restaurant/hawker name + location)
  - **Explanation** (why this choice was made)

---

### 4. Simple Web UI (via Streamlit)
- Web form to input preferences
- Button to generate recommendation
- Output area displaying:
  - Dish recommendation
  - Restaurant/hawker details (simulated or real)
  - Map (optional or stub)
  - Reasoning from LLM

---

## 🗂️ Suggested Project Structure (Kaggle-Compatible)

```plaintext
/AI-Food-Recommender
│
├── recommender_agent.ipynb         # Main notebook for simulation
│
├── data/
│   ├── food_places_sample.csv      # Sample food places (manual or API-based)
│   └── food_tags.csv               # Dishes with tags (diet, cuisine, etc.)
│
├── utils/
│   ├── weather.py                  # Simulate or fetch weather
│   ├── food_places.py              # Place lookup logic (API or static CSV)
│   └── llm_agent.py                # Prompt template and response parsing
│
├── streamlit_app.py (optional)     # Streamlit app for local use
└── README.md
```

---

## 📦 Dependencies (for Kaggle Notebook)

You can install external packages if needed (only if not already available on Kaggle):

```python
!pip install openai streamlit geopy requests
```

---

## 🧪 Example User Scenario

**Input**:
- Preferences: Spicy, Halal
- Time: Dinner
- Location: Petaling Jaya
- Destination: Subang Jaya
- Weather: Rainy

**AI Recommendation Output**:
- **🍛 Dish**: Nasi Kukus Ayam Berempah
- **📍 Place**: Warong Chef Mama, SS15 Subang
- **💬 Reason**: "Given the rainy evening and your preference for spicy halal food, Nasi Kukus is a warm, satisfying option. Warong Chef Mama is on your way and highly rated for this dish."

---

## ✅ Development Phases

| Phase | Task |
|-------|------|
| 1 | Create user input form in notebook |
| 2 | Mock weather + location data modules |
| 3 | Build static food place CSVs (mock Foursquare/Maps API) |
| 4 | Integrate LLM prompt system |
| 5 | Simulate and display results |
| 6 | (Optional) Streamlit version for local testing/demo |

---

## 📘 Notes

- Kaggle doesn’t support Streamlit inside the notebook, but you can simulate all logic using standard input/output.
- You may simulate location/weather APIs using preloaded JSON or CSV files.
- For LLMs, use OpenAI API, HuggingFace, or simulate outputs if API access is limited.

---

## 🧩 Phase 1: User Input Form (Simulated in Notebook)

This section captures user preferences and context needed for the AI food recommender. Since this is a Kaggle notebook, we'll simulate user input using Python variables (not interactive widgets like Streamlit).

The input fields include:
- Preferred flavor/taste (e.g., spicy, sweet)
- Dietary restrictions (e.g., vegetarian, halal)
- Time of day (e.g., breakfast, lunch, dinner)
- Current location (e.g., Bangsar, KL Sentral)
- Destination (optional, e.g., heading to Pavilion KL)
- Food type preference (optional, e.g., street food, café)


In [2]:
# Simulated user input in Kaggle notebook

user_input = {
    "preferred_flavor": "spicy",             # e.g., spicy, sweet, savory, sour
    "dietary_restrictions": ["halal"],       # e.g., vegetarian, halal, gluten-free
    "time_of_day": "lunch",                  # breakfast, lunch, dinner
    "current_location": "KL Sentral",        # starting point
    "destination": "Pavilion KL",            # optional
    "food_type": "street food"               # optional, e.g., street food, healthy, cafe
}

# Print to confirm input
print("✅ User Input Collected:")
for key, value in user_input.items():
    print(f"{key.replace('_', ' ').capitalize()}: {value}")


✅ User Input Collected:
Preferred flavor: spicy
Dietary restrictions: ['halal']
Time of day: lunch
Current location: KL Sentral
Destination: Pavilion KL
Food type: street food


## 🌤️ Phase 2: Get Current Weather Report Based on Location

In this phase, we fetch the current weather using the user's current location. This adds real-time context to the food recommendation (e.g., suggesting hot soup during rainy weather).

Steps:
1. Convert the user's location (e.g., "KL Sentral") into geographic coordinates using Geocoding.
2. Use the coordinates to fetch the current weather from OpenWeatherMap API.


## 🌤️ Fetch Weather Using Secure API Key (Kaggle Secrets)

This phase fetches real-time weather conditions using OpenWeatherMap API. To protect sensitive credentials, we use **Kaggle Secrets** instead of hardcoding the API key.


In [3]:
import requests
from geopy.geocoders import Nominatim
from kaggle_secrets import UserSecretsClient
import os

def fetch_weather(user_input):
    """
    Fetches weather data for the user's current location.
    """
    # Step 1: Geocode current location
    location_name = user_input["current_location"]
    geolocator = Nominatim(user_agent="food-recommender")
    
    print(f"🔍 Looking up coordinates for: {location_name}")
    location = geolocator.geocode(location_name)

    if location:
        lat, lon = location.latitude, location.longitude
        print(f"📍 Coordinates for {location_name}: ({lat}, {lon})")

        # Step 2: Access OpenWeather API key from Kaggle secrets (using environment variable)
        user_secrets = UserSecretsClient()
        api_key = user_secrets.get_secret("OPENWEATHER_API_KEY")
        
        if not api_key:
            raise ValueError("❌ API key not found. Ensure you've added the secret to your notebook settings.")
        
        print("🔑 API key retrieved successfully.")

        # Step 3: Call OpenWeatherMap API
        weather_url = (
            f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric"
        )
        
        print("🌐 Fetching weather data from OpenWeatherMap...")
        response = requests.get(weather_url)

        if response.status_code == 200:
            weather_data = response.json()
            weather_desc = weather_data["weather"][0]["description"]
            temperature = weather_data["main"]["temp"]
            print(f"🌤️ Current weather at {location_name}: {weather_desc}, {temperature}°C")
        else:
            print(f"⚠️ Failed to fetch weather data. Response code: {response.status_code}")
    else:
        print(f"⚠️ Could not find coordinates for location: {location_name}")


if __name__ == "__main__":
    # Example user input
    user_input = {
        "current_location": "Kuala Lumpur"
    }

    # Fetch weather information
    fetch_weather(user_input)

🔍 Looking up coordinates for: Kuala Lumpur
📍 Coordinates for Kuala Lumpur: (3.1526589, 101.7022205)
🔑 API key retrieved successfully.
🌐 Fetching weather data from OpenWeatherMap...
🌤️ Current weather at Kuala Lumpur: few clouds, 33.78°C


## 📍 Phase 3: Dynamic Food Place Recommendation using Google Places API

In this phase, we expand the user input form to collect dietary preferences, food types, and travel context. Then, we connect to the **Google Places API** to dynamically suggest food venues near the user’s *destination* based on these criteria.

This hybrid approach simulates how an AI agent can generate contextualized and culturally relevant food recommendations — a critical step toward building a real-time decision support system for Malaysian and Asian cuisine lovers.

### 👣 Steps
1. Collect user preferences and travel context (e.g. location, destination).
2. Use Google Places API to search for food venues near the destination.
3. Display the top matched venues in a table.


In [4]:
# 📥 User Input Form
import ipywidgets as widgets
from IPython.display import display

# Define input widgets
dietary_input = widgets.Dropdown(
    options=["halal", "vegetarian", "vegan", "no preference"],
    description="Dietary:"
)

food_type_input = widgets.Text(
    value="street food",
    description="Food Type:"
)

current_location_input = widgets.Text(
    value="KL Sentral",
    description="Current:"
)

destination_input = widgets.Text(
    value="Pavilion KL",
    description="Destination:"
)

submit_button = widgets.Button(description="🔍 Get Recommendations")

# Display the widgets
display(dietary_input, food_type_input, current_location_input, destination_input, submit_button)

# Store input when user clicks button
user_input = {}

def on_submit_clicked(b):
    user_input["dietary"] = dietary_input.value
    user_input["food_type"] = food_type_input.value
    user_input["current_location"] = current_location_input.value
    user_input["destination"] = destination_input.value
    print("✅ Input received:", user_input)

submit_button.on_click(on_submit_clicked)


Dropdown(description='Dietary:', options=('halal', 'vegetarian', 'vegan', 'no preference'), value='halal')

Text(value='street food', description='Food Type:')

Text(value='KL Sentral', description='Current:')

Text(value='Pavilion KL', description='Destination:')

Button(description='🔍 Get Recommendations', style=ButtonStyle())

In [15]:
import requests
from kaggle_secrets import UserSecretsClient
import pandas as pd

def search_google_places(user_input, radius=3000):
    """
    Query Google Places API based on user input for food recommendations.
    """
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("GOOGLE_PLACES_API_KEY")
    
    query = f"{user_input['dietary']} {user_input['food_type']} food"
    location = user_input['destination']

    print(f"📡 Searching for: '{query}' near '{location}'")

    url = (
        "https://maps.googleapis.com/maps/api/place/textsearch/json"
        f"?query={query}+restaurant+in+{location}"
        f"&radius={radius}&key={api_key}"
    )

    response = requests.get(url)
    results = []

    if response.status_code == 200:
        data = response.json()
        for place in data.get("results", []):
            results.append({
                "name": place.get("name"),
                "address": place.get("formatted_address"),
                "rating": place.get("rating"),
                "total_reviews": place.get("user_ratings_total"),
                "types": ", ".join(place.get("types", []))
            })
    else:
        print(f"❌ Google Places API Error: {response.status_code}")
    
    return pd.DataFrame(results)


In [16]:
# ⚠️ Make sure user_input is filled by submitting the form first
if user_input:
    recommended_places = search_google_places(user_input)
    if not recommended_places.empty:
        display(recommended_places.head(5))
    else:
        print("😕 No suitable places found.")
else:
    print("⚠️ Please fill and submit the user input form first.")


📡 Searching for: 'halal street food food' near 'Pavilion KL'


,name,address,rating,total_reviews,types
0,Ei8ht Avenue @ Pavilion KL,"Lot 1.29.01 - Lot 1.29.04, Pavilion KL, 168, J...",3.4,187,"food, point_of_interest, establishment"
1,Food Republic at Pavilion KL,"Lot 1.41.00 - 1.51.00 & P1.13.00 - P1.20.00, 1...",4.2,3154,"food, point_of_interest, establishment"
2,Mee Tarik @ Food Republic,"Pavilion, Level 1, 168, Jln Bukit Bintang, Buk...",4.5,40,"food, point_of_interest, establishment"
3,ParaThai @ Pavilion KL,"Lot 1.03 & 1.04, Level 1, Pavilion KL, 168, Jl...",4.8,2329,"restaurant, food, point_of_interest, establish..."
4,Madam Kwan's,"00, Level 1, Lot 1.16, 168, Jln Bukit Bintang,...",4.4,4421,"restaurant, food, point_of_interest, establish..."


## 🌦️ Phase 4: Weather-Aware Food Recommendations

In this phase, we enhance the intelligence of our AI food agent by factoring in real-time **weather conditions** at the user's **destination**. The logic adapts the recommendation by associating certain food types with weather patterns. For example:

- ☀️ Hot weather → Iced desserts or refreshing beverages
- ☁️ Cloudy → Grilled or fried items
- 🌧️ Rainy → Hot soups or cozy indoor cafes

This makes the recommendation more human-like and context-aware, improving user satisfaction and relevance.

We’ll use:
- **OpenWeatherMap API** to get live weather info.
- Google Places API (from Phase 3) to fetch updated results.
- Optional fallback logic if weather data is unavailable.


In [17]:
from geopy.geocoders import Nominatim
import requests
from kaggle_secrets import UserSecretsClient

def fetch_weather_at_location(location_name):
    """
    Fetch current weather data for a given location using OpenWeatherMap API.
    """
    geolocator = Nominatim(user_agent="food-recommender")
    location = geolocator.geocode(location_name)

    if not location:
        print(f"⚠️ Could not geolocate destination: {location_name}")
        return None

    lat, lon = location.latitude, location.longitude

    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("OPENWEATHER_API_KEY")

    url = (
        f"https://api.openweathermap.org/data/2.5/weather?"
        f"lat={lat}&lon={lon}&appid={api_key}&units=metric"
    )

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        weather = {
            "description": data["weather"][0]["description"],
            "temperature": data["main"]["temp"]
        }
        print(f"🌡️ Weather at {location_name}: {weather['description']}, {weather['temperature']}°C")
        return weather
    else:
        print(f"❌ Weather API error: {response.status_code}")
        return None


In [18]:
def recommend_places_with_weather(user_input):
    """
    Enhances recommendations by factoring in destination weather.
    """
    # Step 1: Fetch destination weather
    weather = fetch_weather_at_location(user_input["destination"])
    
    if weather:
        weather_desc = weather["description"].lower()
        
        # Step 2: Inject weather-related food preference
        if "rain" in weather_desc:
            weather_bias = "hot soup"
        elif "clear" in weather_desc or "sun" in weather_desc:
            weather_bias = "iced dessert"
        elif "cloud" in weather_desc:
            weather_bias = "grilled food"
        else:
            weather_bias = "indoor dining"

        print(f"🌥️ Weather-based suggestion: Adding '{weather_bias}' to food type.")
        user_input["food_type"] += f" {weather_bias}"
    else:
        print("⚠️ No weather adjustment applied (fallback to original query).")

    # Step 3: Proceed to search with adjusted query
    return search_google_places(user_input)


In [19]:
# 🚀 Full recommendation with weather-aware logic
if user_input:
    full_results = recommend_places_with_weather(user_input)
    if not full_results.empty:
        display(full_results.head(5))
    else:
        print("😕 No suitable places found.")
else:
    print("⚠️ Please fill and submit the user input form first.")


🌡️ Weather at Pavilion KL: few clouds, 33.74°C
🌥️ Weather-based suggestion: Adding 'grilled food' to food type.
📡 Searching for: 'halal street food grilled food food' near 'Pavilion KL'


,name,address,rating,total_reviews,types
0,Serai@Pavilion,"Lot 7 . 01 . 04, 168, Jln Bukit Bintang, Bukit...",4.7,3904,"restaurant, food, point_of_interest, establish..."
1,Food Republic at Pavilion KL,"Lot 1.41.00 - 1.51.00 & P1.13.00 - P1.20.00, 1...",4.2,3154,"food, point_of_interest, establishment"
2,ParaThai @ Pavilion KL,"Lot 1.03 & 1.04, Level 1, Pavilion KL, 168, Jl...",4.8,2329,"restaurant, food, point_of_interest, establish..."
3,探鱼烤鱼 - TANYU Pavilion Kuala Lumpur,"Lot 7.01.02, Pavilion Kuala Lumpur, 168, Jln B...",4.8,3245,"restaurant, food, point_of_interest, establish..."
4,TGI Fridays Pavilion KL,"Lot 6 . 01 . 07 & 6 . 01 . 08, 168, Jln Bukit ...",4.5,3137,"restaurant, food, point_of_interest, establish..."


## 🧠 Phase 5: Summarize Food Recommendations Using LLM (OpenAI GPT) + Fallback

In this final phase, we generate a **natural language summary** of the top food recommendations using a Language Model (LLM). This humanizes the result and gives users a more conversational and intuitive experience. However, to make our system more robust and demo-ready, we include a fallback simulated summary generator in case:
- OpenAI API key is missing
- API quota is exceeded
- Any other failure occurs
- 
For this prototype, we will use **OpenAI’s GPT API** (or other hosted LLMs via HTTP request) to:
- Highlight the top 3 places
- Describe their food type and location
- Suggest one standout recommendation

🔐 You’ll need an **OpenAI API key** saved in Kaggle secrets under `OPENAI_API_KEY`.


This ensures our AI Agent can still respond meaningfully, even when LLM services are unavailable.


In [20]:
!pip install -q openai

In [21]:
from openai import OpenAI
from kaggle_secrets import UserSecretsClient

# Initialize OpenAI client
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

def generate_summary(df, user_input):
    """
    Generate a summary using OpenAI GPT.
    Falls back to a simulated version if API fails.
    """
    top_places = df.head(3).to_dict(orient="records")
    food_type = user_input.get("food_type", "delicious food")
    destination = user_input.get("destination", "your destination")

    places_text = "\n".join([
        f"{i+1}. {place['name']} ({place['rating']}⭐) - {place['address']}"
        for i, place in enumerate(top_places)
    ])

    prompt = f"""
    You are a friendly food guide. A user is looking for {food_type} in {destination}.
    Here are the top recommended places:
    
    {places_text}
    
    Please write a short paragraph summarizing these recommendations. Mention the highlights and suggest one must-try.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful food recommendation assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=300
        )
        summary = response.choices[0].message.content.strip()
        return summary

    except Exception as e:
        print(f"❌ Error calling OpenAI API: {e}")
        print("⚠️ Falling back to simulated summary...\n")
        return generate_summary_simulated(df, user_input)

def generate_summary_simulated(df, user_input):
    """
    Simulated summary generator (no LLM required).
    """
    top_places = df.head(3).to_dict(orient="records")
    food_type = user_input.get("food_type", "delicious food")
    destination = user_input.get("destination", "your destination")

    summary = f"Here are the top picks for {food_type} near {destination}:\n"

    for i, place in enumerate(top_places, 1):
        summary += f"{i}. {place['name']} ({place['rating']}⭐) located at {place['address']}.\n"

    summary += f"\n🍽️ Based on reviews, we recommend trying {top_places[0]['name']} first!"

    return summary


In [22]:
# Final pipeline
if user_input:
    full_results = recommend_places_with_weather(user_input)

    if not full_results.empty:
        display(full_results.head(3))

        print("\n📝 Generating summary...\n")
        summary = generate_summary(full_results, user_input)

        if summary:
            print("🧾 Summary:\n")
            print(summary)
        else:
            print("⚠️ Failed to generate summary.")
    else:
        print("😕 No suitable places found.")
else:
    print("⚠️ Please fill and submit the user input form first.")



🌡️ Weather at Pavilion KL: few clouds, 33.77°C
🌥️ Weather-based suggestion: Adding 'grilled food' to food type.
📡 Searching for: 'halal street food grilled food grilled food food' near 'Pavilion KL'


,name,address,rating,total_reviews,types
0,Ei8ht Avenue @ Pavilion KL,"Lot 1.29.01 - Lot 1.29.04, Pavilion KL, 168, J...",3.4,187,"food, point_of_interest, establishment"
1,Halab KL Arabic Restaurant,"35, Jalan Berangan, Bukit Bintang, 50200 Kuala...",4.5,4334,"restaurant, food, point_of_interest, establish..."
2,Serai@Pavilion,"Lot 7 . 01 . 04, 168, Jln Bukit Bintang, Bukit...",4.7,3904,"restaurant, food, point_of_interest, establish..."



📝 Generating summary...

❌ Error calling OpenAI API: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
⚠️ Falling back to simulated summary...

🧾 Summary:

Here are the top picks for street food grilled food grilled food near Pavilion KL:
1. Ei8ht Avenue @ Pavilion KL (3.4⭐) located at Lot 1.29.01 - Lot 1.29.04, Pavilion KL, 168, Jln Bukit Bintang, Bukit Bintang, 55100 Kuala Lumpur, Wilayah Persekutuan Kuala Lumpur, Malaysia.
2. Halab KL Arabic Restaurant (4.5⭐) located at 35, Jalan Berangan, Bukit Bintang, 50200 Kuala Lumpur, Wilayah Persekutuan Kuala Lumpur, Malaysia.
3. Serai@Pavilion (4.7⭐) located at Lot 7 . 01 . 04, 168, Jln Bukit Bintang, Bukit Bintang, 55100 Kuala Lumpur, Federal Territory of Kuala Lumpur, Malaysia.

🍽️ 

In [ ]:
full_results.head()

## 📍 Phase 6: Display Recommended Locations on an Interactive Map
**📌 Objective**
In this phase, we enhance the user experience by plotting the top recommended food spots on an interactive map using Folium. This allows users to visualize where each recommended location is, making it easier to decide based on proximity or direction of travel.

**🧰 Tools Used**
- Folium: For generating interactive maps in notebooks.
- Pandas: For handling place data.
- Geopy: To geocode any locations if needed.

In [ ]:
import requests
import pandas as pd
from kaggle_secrets import UserSecretsClient

def get_google_places(user_input):
    """
    Fetch recommended nearby places to eat using Google Places API.
    """
    # Step 1: Get coordinates for current location
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="food-recommender-places")
    location = geolocator.geocode(user_input["current_location"])

    if not location:
        print(f"⚠️ Could not geocode location: {user_input['current_location']}")
        return pd.DataFrame()

    lat, lon = location.latitude, location.longitude
    keyword = user_input.get("preferences", "restaurant")

    # Step 2: Fetch API key from Kaggle Secrets
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("GOOGLE_PLACES_API_KEY")

    if not api_key:
        raise ValueError("❌ Google Places API Key not found in Kaggle Secrets.")

    # Step 3: Make API call
    url = (
        f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        f"?location={lat},{lon}"
        f"&radius=3000"
        f"&type=restaurant"
        f"&keyword={keyword}"
        f"&key={api_key}"
    )

    response = requests.get(url)

    if response.status_code != 200:
        print("❌ Error calling Google Places API.")
        return pd.DataFrame()

    places = response.json().get("results", [])

    # Step 4: Build places data list
    places_data = []
    for place in places:
        places_data.append({
            "name": place.get("name", "N/A"),
            "address": place.get("vicinity", "N/A"),
            "rating": place.get("rating", "N/A"),
            "lat": place["geometry"]["location"]["lat"],
            "lon": place["geometry"]["location"]["lng"]
        })

    # Convert to DataFrame
    df = pd.DataFrame(places_data)

    if df.empty:
        print("⚠️ No nearby places found.")
    else:
        print(f"✅ Found {len(df)} recommended places.")

    return df

In [ ]:
# Run this to get full results ready for mapping
print("📡 Retrieving recommended food places...\n")
full_results = get_google_places(user_input)

# Optional preview
full_results.head()

In [23]:
import folium
from folium.plugins import MarkerCluster

def plot_recommendations_map(df, user_input):
    """
    Plots top recommended places on an interactive map.
    """
    if df.empty:
        print("⚠️ No data to plot.")
        return

    # Get starting center point based on user's current location
    location_name = user_input.get("current_location", "Kuala Lumpur")
    geolocator = Nominatim(user_agent="food-recommender-map")
    location = geolocator.geocode(location_name)

    if location:
        start_coords = [location.latitude, location.longitude]
    else:
        start_coords = [3.1390, 101.6869]  # fallback to KL center

    # Create folium map
    food_map = folium.Map(location=start_coords, zoom_start=13)
    marker_cluster = MarkerCluster().add_to(food_map)

    for _, row in df.iterrows():
        lat = row["lat"]
        lon = row["lon"]
        name = row["name"]
        rating = row["rating"]
        address = row["address"]

        popup_html = f"<b>{name}</b><br>⭐ Rating: {rating}<br>{address}"
        folium.Marker(
            location=[lat, lon],
            popup=popup_html,
            icon=folium.Icon(color="red", icon="cutlery", prefix="fa")
        ).add_to(marker_cluster)

    return food_map


In [24]:
# Display recommendations on map
print("🗺️ Plotting recommended places on map...\n")
map_display = plot_recommendations_map(full_results, user_input)

if map_display:
    display(map_display)
else:
    print("⚠️ Could not render the map.")


🗺️ Plotting recommended places on map...



KeyError: 'lat'